# Reactome

Created by: Charles Dai <br>
Credit to: Moshe Silverstein

Data Source: http://reactome.org/pages/download-data/

In [1]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [2]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [3]:
# from clustergrammer_widget import *
# net = Network(clustergrammer_widget)

In [4]:
%load_ext autoreload
%autoreload 2

### Notebook Information

In [5]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

This notebook was run on: 2020-06-29 
Python version: 3.8.0 (default, Oct 28 2019, 16:14:01) 
[GCC 8.3.0]


# Initialization

### Load Mapping Dictionaries

In [6]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

Gathering sources: 100%|██████████| 3/3 [00:09<00:00,  3.23s/it]


### Output Path

In [7]:
output_name = 'reactome'

path = 'Output/Reactome'
if not os.path.exists(path):
    os.makedirs(path)

In [8]:
%%appyter hide_code
{% do SectionField(
    name='data',
    title='Load Data',
    subtitle='Upload Files from the Reactome Pathways Database',
) %}

# Load Data

In [15]:
%%appyter code_exec

df = pd.read_csv({{FileField(
    constraint='.*\.zip$',
    name='pathways_gene', 
    label='Pathways Gene Set (gmt.zip)', 
    default='Input/Reactome/ReactomePathways.gmt.zip',
    section='data')
}}, sep='%', header=None)

```python

df = pd.read_csv('Input/Reactome/ReactomePathways.gmt.zip', sep='%', header=None)
```

In [16]:
df.head()

,0
0,2-LTR circle formation\tR-HSA-164843\tBANF1\tH...
1,5-Phosphoribose 1-diphosphate biosynthesis\tR-...
2,A tetrasaccharide linker sequence is required ...
3,ABC transporter disorders\tR-HSA-5619084\tABCA...
4,ABC transporters in lipid homeostasis\tR-HSA-1...


In [17]:
df.shape

(2400, 1)

# Pre-process Data

## Separate and Split Gene List

In [12]:
df[0], df[1] = df[0].str.split('\t').str[0], df[0].str.split('\t').str[1:]
df.columns=['Pathway', 'Gene Symbol']
df.head()

,Pathway,Gene Symbol
0,2-LTR circle formation,"[R-HSA-164843, BANF1, HMGA1, LIG4, PSIP1, XRCC..."
1,5-Phosphoribose 1-diphosphate biosynthesis,"[R-HSA-73843, PRPS1, PRPS1L1, PRPS2]"
2,A tetrasaccharide linker sequence is required ...,"[R-HSA-1971475, AGRN, B3GALT6, B3GAT1, B3GAT2,..."
3,ABC transporter disorders,"[R-HSA-5619084, ABCA1, ABCA12, ABCA3, ABCB11, ..."
4,ABC transporters in lipid homeostasis,"[R-HSA-1369062, ABCA10, ABCA12, ABCA2, ABCA3, ..."


In [13]:
df = df.explode('Gene Symbol')
df = df.set_index('Gene Symbol')
df.head()

,Pathway
Gene Symbol,
R-HSA-164843,2-LTR circle formation
BANF1,2-LTR circle formation
HMGA1,2-LTR circle formation
LIG4,2-LTR circle formation
PSIP1,2-LTR circle formation


# Filter Data

## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [14]:
df = uf.map_symbols(df, symbol_lookup, remove_duplicates=True)
df.shape

100%|██████████| 118714/118714 [00:00<00:00, 579593.74it/s]


(113787, 1)

# Analyze Data

## Create Binary Matrix

In [ ]:
binary_matrix = uf.binary_matrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.save_data(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

## Create Gene List

In [ ]:
gene_list = uf.gene_list(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.save_data(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

## Create Attribute List

In [ ]:
attribute_list = uf.attribute_list(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.save_data(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

## Create Gene and Attribute Set Libraries

In [ ]:
uf.save_setlib(binary_matrix, 'gene', 'up', path, output_name + '_gene_up_set')

In [ ]:
uf.save_setlib(binary_matrix, 'attribute', 'up', path, 
                           output_name + '_attribute_up_set')

## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.similarity_matrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.save_data(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

In [ ]:
# net.load_df(attribute_similarity_matrix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.similarity_matrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.save_data(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene-Attribute Edge List

In [ ]:
edge_list = uf.edge_list(binary_matrix)
uf.save_data(edge_list, path, output_name + '_edge_list', 
        ext='tsv', compression='gzip')

# Create Downloadable Save File

In [ ]:
uf.archive(path)

### Link to download output files: [click here](./output_archive.zip)